# Time Series Forecasting

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import dates
import seaborn as sns
import joblib
import statsmodels.api as sm
# import bamboolib as bam

from tkinter import *
import dtale as dt
import plotly.graph_objects as go

# import glob

# from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller

from sklearn import metrics
from sklearn.metrics import  mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

pd.set_option('display.float_format', '{:,.3}'.format)
# pd.set_option('max_columns', 100)


/home/fqp/anaconda3/envs/s1_v3/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [ ]:
# Standard imports 
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, output_file, save

from bokeh.models.tools import HoverTool
from bokeh.models import Legend, ColumnDataSource, Range1d
output_notebook()

In [2]:

#* Ignore Warnings
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

### 1.] Preparations

#### Loading & Preparing the Data

In [3]:
df_sys = joblib.load("./data/GTMA Trades_df_sys")
df_enr = joblib.load("./data/GTMA Trades_df_enr")

# df_sys.head(6)

In [4]:

#? Resampling to Daily frequency.
df_sys_D1 = df_sys.set_index(['End Time_D&T'])[['Volume', 'Cost']].resample('D').sum()
df_sys_D2 = df_sys.set_index(['End Time_D&T'])[['Price', 'Trade Hours Duration']].resample('D').mean()
df_sys_D = df_sys_D1.join(df_sys_D2,how='left')
# df_sys_D.head(4)

#ToDo: PowerTransformer here after Resampling.

In [5]:
df_enr_D1 = df_enr.set_index(['End Time_D&T'])[['Volume', 'Cost']].resample('D').sum()
df_enr_D2 = df_enr.set_index(['End Time_D&T'])[['Price', 'Trade Hours Duration']].resample('D').mean()
df_enr_D = df_enr_D1.join(df_enr_D2,how='left')
# df_enr_D.head(4)

In [ ]:

#? Using DTale to remove rows from 2018 and older.
#ToDo: CONSIDER: Remove 2018 data before Resampling?
dt.show(df_sys_D, open_browser=True, ignore_index=True)

In [ ]:
dt.show(df_enr_D, open_browser=True, ignore_index=True)

In [ ]:

#? Reloading the .csv after removing rows
df_sys_D = pd.read_csv("./data/df_sysD_19-21.csv")
df_sys_D.head(4)

In [ ]:
df_enr_D = pd.read_csv("./data/df_enrD_19-21.csv")
joblib.dump(df_enr_D,"./data/GTMA Trades_df_enrD_19-21")

In [ ]:
joblib.dump(df_sys_D,"./data/GTMA Trades_df_sysD_19-21")

In [7]:
df_sysD_19to21 = joblib.load('./data/GTMA Trades_df_sysD_19-21')
# df_sysD_19to21.head(4)

In [8]:
df_enrD_19to21 = joblib.load('./data/GTMA Trades_df_enrD_19-21')

> FORMATTED DF

In [9]:

df_sysD_19to21["End Time_D&T"] = pd.to_datetime(df_sysD_19to21["End Time_D&T"], format="%Y-%m-%d")
# df_sysD_19to21.head(6)

In [ ]:

#? Checking datatypes.
df_sysD_19to21.dtypes

In [10]:
df_enrD_19to21["End Time_D&T"] = pd.to_datetime(df_enrD_19to21["End Time_D&T"], format="%Y-%m-%d")

In [6]:
transformer_pwr = joblib.load('./transformer_pwr')

In [14]:
transformer_pwr.fit_transform(df_sysD_19to21["Cost"])

ValueError: Expected 2D array, got 1D array instead:
array=[ 3.44841000e+05 -3.14034000e+04  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -1.25268000e+04  7.07850200e+04  1.86893250e+05
 -1.14789900e+05 -5.16240000e+04 -2.91968000e+04  2.74789300e+05
  9.54153550e+05  4.77503600e+05 -1.21961900e+05  3.83639600e+05
 -5.32954924e+04  0.00000000e+00 -2.70120000e+03  0.00000000e+00
  5.75852520e+05 -1.52358000e+04  0.00000000e+00  0.00000000e+00
  0.00000000e+00  1.93785400e+05 -7.00206000e+04 -1.63727900e+05
 -4.03200000e+03  2.06640000e+05  0.00000000e+00 -4.44600000e+03
  1.39768200e+05  1.16678200e+05 -3.48443200e+04  2.39556000e+05
  6.19158550e+04  7.47075000e+04 -1.35828450e+04  7.53200950e+05
  4.88373200e+04 -2.69970000e+04  1.02937310e+05 -9.65376000e+04
  2.97266825e+05  5.66373000e+04  4.04364310e+05  5.11671000e+04
  1.76688040e+05  4.95929300e+04  2.82497350e+04 -1.55408600e+05
 -1.49968650e+05  1.72472000e+04  2.36286250e+04  8.57314000e+04
  8.47792000e+04 -1.37043000e+0

In [17]:

# standardized = transformer_pwr.fit_transform(df_sysD_19to21["Cost"]).reshape(-1)
df_sysD_19to21["Cost_pt"] = transformer_pwr.fit_transform(df_sysD_19to21[["Cost"]]).reshape(-1)

# standardized = transformer_pwr.fit_transform(df_enrD_19to21["Cost"]).reshape(-1)
df_enrD_19to21["Cost_pt"]= transformer_pwr.fit_transform(df_enrD_19to21[["Cost"]]).reshape(-1)

#### Auxillary Functions

In [18]:
dict_train = {
        "Q1" : 365,            #? Start of 2020
        "Q2" : 365 + 89,       #? Start of March
        "Q3" : 365 + 182,      #? 2020 midyear, Start of July     
        "Q4" : 365 + 182 + 92  #? Start of Oct.
        }

layout = go.Layout(
        margin=go.layout.Margin(
        l=20, #left margin
        r=15, #right margin
        b=15, #bottom margin
        t=35 ),#top margin
        xaxis_title="Date", yaxis_title="Cost (£)")

In [19]:

def go_plotter(df_train, df_test, df_forecast, FC, LABEL, CAT):
        fig = go.Figure(layout=layout)
        fig.update_layout(title= str(CAT) + " Trades " + str(FC) + "-Day Forecast" + " -- " + LABEL)

        #? Note: Index of incoming dataframes are Datetime
        for i, t in zip([df_train, df_test, df_forecast],["Train","Test","Forecast"]):
                mode_var = 'lines+markers' if (t =="Forecast") else 'lines'
                line_dash = 'dot' if (t =="Train") else 'solid'
                fig.add_trace(go.Scatter(x=i.index, y=i["Cost_pt"].values, name=t, mode=mode_var,
                line_dash=line_dash, hovertemplate="Date = %{x}<br>Cost_pt = %{y}"))
        # fig.for_each_trace(lambda trace: trace.update(fill='tonextx') if trace.name == "Forecast" else ())
        fig.show()

In [20]:
EPSILON = 1e-10
def maape(actual, predicted):
    #? Mean Arctangent Absolute Percentage Error
    #? Note: result is NOT multiplied by 100
    return np.mean(np.arctan(np.abs((actual - predicted) / (actual + EPSILON))))

In [ ]:

#ToDo: Not used. Needs to be updated.
def bokeh_plotter(train,test,forecast):
    #? Specific only for this project: Because of rigidly-named variables.
    
    #? ds_ == "data source", as required by bokeh plots
    ds_train = ColumnDataSource(train)
    ds_test = ColumnDataSource(test)
    ds_forecast = ColumnDataSource(forecast)

    TOOLTIPS = [('Date', '@{End Time_D&T}{%F}'),
                ('Cost', '@Cost{($ 0.00 a)}')]
    
    hover = HoverTool(tooltips=TOOLTIPS, 
                    formatters={
                        '@{End Time_D&T}' : 'datetime',
                            },
                    mode="mouse")
    
    #? Establishing plot
    p = figure(x_axis_type="datetime", plot_width=1100, plot_height=420)
    p.add_tools(hover)

    #? Plot configurations
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_alpha=0.5
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = 'Cost'
    
    #* Main plots
    line_train = p.line(x='End Time_D&T', y='Cost', source=ds_train,color="grey", line_alpha=.35)
    line_test = p.line(x='End Time_D&T', y='Cost', source=ds_test,color="orange", line_dash="dotted",legend_label="Test",line_width=2)
    line_forecast = p.line(x='End Time_D&T', y='Cost', source=ds_forecast, legend_label="ExpoSmooth Forecast -- System Trade")
    
    #? Configs must be AFTER Main plots 
    p.toolbar.autohide = True
    p.legend.click_policy="hide"

    show(p)

### 2.] Exponential Smoothing

#### System Trades

In [64]:
# CAT = "System"
LABEL = "Q1"

FC = 60

df_train = df_sysD_19to21[['End Time_D&T','Cost_pt']][dict_train.get(LABEL)-365:-152].set_index("End Time_D&T", drop=True)
df_test = df_sysD_19to21[['End Time_D&T',"Cost_pt"]].set_index("End Time_D&T", drop=True)
df_test = df_test[["Cost_pt"]][df_train.index[-1]+timedelta(days=1): (df_train.index[-1] + timedelta(days=(FC)))]


In [65]:

#* Daily Frequency

#? Optional arguments: seasonal_periods=30, seasonal='add'
model_fit_sysD = ExponentialSmoothing(df_train, trend='add', damped_trend=True, freq='D',
                                initialization_method="estimated",seasonal_periods=365, seasonal='add').fit(optimized=True)

df_forecast = pd.DataFrame(model_fit_sysD.forecast(len(df_test.index)).values, 
                        index=df_test.index, columns=['Cost_pt'])

# fig.update_layout(title="System Trades " + str(FC) + "-Day Forecast" + " -- " + LABEL)


In [66]:
go_plotter(df_train, df_test, df_forecast, FC, LABEL, "System")
maape(df_test.values, df_forecast.values)

0.7767950011682095

In [ ]:

#? Basis for Bokeh plotting
model_fit_sysD = ExponentialSmoothing(df_sysD_19to21['Cost'][(365+182+1):-152], seasonal_periods=30, trend='add', seasonal='add',
                                    damped_trend=True, initialization_method="heuristic").fit(optimized=True)

df_forecast = pd.DataFrame(model_fit_sysD.forecast(60).values, 
                            index=df_sysD_19to21["End Time_D&T"][-152:-92], columns=['Cost'])

bokeh_plotter(df_train,
            df_test, 
            df_forecast)

#### Energy Trades

In [76]:

LABEL = "Q1"

FC = 60

df_train = df_enrD_19to21[['End Time_D&T','Cost_pt']][dict_train.get(LABEL)-365:-152].set_index("End Time_D&T", drop=True)
df_test = df_enrD_19to21[['End Time_D&T',"Cost_pt"]].set_index("End Time_D&T", drop=True)
df_test = df_test[["Cost_pt"]][df_train.index[-1]+timedelta(days=1): (df_train.index[-1] + timedelta(days=(FC)))]

In [77]:

#? Optional arguments: seasonal_periods=30, seasonal='add'
model_fit_sysD = ExponentialSmoothing(df_train, trend='add', damped_trend=True, freq='D',
                                initialization_method="heuristic",seasonal_periods=365, seasonal='add').fit(optimized=True)

df_forecast = pd.DataFrame(model_fit_sysD.forecast(len(df_test.index)).values, 
                        index=df_test.index, columns=['Cost_pt'])


In [78]:
go_plotter(df_train, df_test, df_forecast, FC, LABEL, "Energy")
maape(df_test.values, df_forecast.values)

0.8660379006501667

_______________________

### References

1.] MAAPE -- "Thus, MAAPE is scale-independent, can be interpreted intuitively as an absolute percentage error, and is simple to calculate." [https://www.sciencedirect.com/science/article/pii/S0169207016000121]

2.] Forecasting metrics -- forecasting_metrics.py, https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9

3.] Ignoring warnings -- https://stackoverflow.com/questions/34444607/how-to-ignore-statsmodels-maximum-likelihood-convergence-warning